Descargar dependencias


In [30]:

!pip install numpy scikit-fuzzy gradio clipspy

print("ok")


ok


Importar las librerias

In [31]:
import numpy as np
import skfuzzy as fuzz
import skfuzzy.control as ctrl
import clips
import gradio as gr

In [32]:
edad = ctrl.Antecedent(np.arange(10, 100, 1), 'edad')
estado_animo = ctrl.Antecedent(np.arange(0, 11, 1), 'estado_animo')
actividad = ctrl.Antecedent(np.arange(0, 11, 1), 'actividad')
dia_semana = ctrl.Antecedent(np.arange(0, 7, 1), 'dia_semana')
recomendacion = ctrl.Consequent(np.arange(0, 11, 1), 'recomendacion')

In [33]:
# Edad (joven, adulto, mayor)
edad['joven'] = fuzz.trimf(edad.universe, [10, 18, 30])
edad['adulto'] = fuzz.trimf(edad.universe, [25, 40, 60])
edad['mayor'] = fuzz.trimf(edad.universe, [50, 70, 100])

# Estado de ánimo (bajo, medio, alto)
estado_animo['bajo'] = fuzz.trimf(estado_animo.universe, [0, 0, 5])
estado_animo['medio'] = fuzz.trimf(estado_animo.universe, [3, 5, 7])
estado_animo['alto'] = fuzz.trimf(estado_animo.universe, [5, 10, 10])

# Actividad (baja, media, alta)
actividad['baja'] = fuzz.trimf(actividad.universe, [0, 0, 5])
actividad['media'] = fuzz.trimf(actividad.universe, [3, 5, 7])
actividad['alta'] = fuzz.trimf(actividad.universe, [5, 10, 10])

# Día de la semana (entre semana, fin de semana)
dia_semana['entre_semana'] = fuzz.trimf(dia_semana.universe, [0, 2, 4])
dia_semana['fin_de_semana'] = fuzz.trimf(dia_semana.universe, [5, 6, 6])

# Recomendación (baja, media, alta)
recomendacion['baja'] = fuzz.trimf(recomendacion.universe, [0, 0, 5])
recomendacion['media'] = fuzz.trimf(recomendacion.universe, [3, 5, 7])
recomendacion['alta'] = fuzz.trimf(recomendacion.universe, [5, 10, 10])

In [34]:
regla1 = ctrl.Rule(estado_animo['bajo'] & actividad['baja'], recomendacion['baja'])
regla2 = ctrl.Rule(estado_animo['alto'] | actividad['alta'], recomendacion['alta'])
regla3 = ctrl.Rule(edad['mayor'] & estado_animo['medio'], recomendacion['media'])
regla4 = ctrl.Rule(dia_semana['fin_de_semana'] & actividad['media'], recomendacion['alta'])

In [35]:
sistema_ctrl = ctrl.ControlSystem([regla1, regla2, regla3, regla4])
sistema = ctrl.ControlSystemSimulation(sistema_ctrl)

In [36]:
def obtener_recomendacion(edad_input, estado_animo_input, actividad_input, dia_semana_input):
    dias_mapping = {'lunes': 0, 'martes': 1, 'miércoles': 2, 'jueves': 3, 'viernes': 4, 'sábado': 5, 'domingo': 6}

    sistema.input['edad'] = edad_input
    sistema.input['estado_animo'] = estado_animo_input
    sistema.input['actividad'] = actividad_input
    sistema.input['dia_semana'] = dias_mapping[dia_semana_input]

    sistema.compute()
    resultado = sistema.output['recomendacion']
    return f"Nivel de recomendación: {round(resultado, 2)}"




In [37]:
interfaz = gr.Interface(
    fn=obtener_recomendacion,
    inputs=[
        gr.Number(label='Edad', value=25),
        gr.Slider(0, 10, label='Estado de Ánimo'),
        gr.Slider(0, 10, label='Actividad'),
        gr.Dropdown(choices=['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo'], label='Día de la Semana')
    ],
    outputs="text"
)

interfaz.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5f770e4c7d343b49ad.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
